In [1]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import seaborn as sns
import numpy as np

C:\Users\amogh\AppData\Roaming\Python\Python310\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
iris = load_iris()
X = iris.data
y = iris.target

In [3]:
X, y

(array([[5.1, 3.5, 1.4, 0.2],
        [4.9, 3. , 1.4, 0.2],
        [4.7, 3.2, 1.3, 0.2],
        [4.6, 3.1, 1.5, 0.2],
        [5. , 3.6, 1.4, 0.2],
        [5.4, 3.9, 1.7, 0.4],
        [4.6, 3.4, 1.4, 0.3],
        [5. , 3.4, 1.5, 0.2],
        [4.4, 2.9, 1.4, 0.2],
        [4.9, 3.1, 1.5, 0.1],
        [5.4, 3.7, 1.5, 0.2],
        [4.8, 3.4, 1.6, 0.2],
        [4.8, 3. , 1.4, 0.1],
        [4.3, 3. , 1.1, 0.1],
        [5.8, 4. , 1.2, 0.2],
        [5.7, 4.4, 1.5, 0.4],
        [5.4, 3.9, 1.3, 0.4],
        [5.1, 3.5, 1.4, 0.3],
        [5.7, 3.8, 1.7, 0.3],
        [5.1, 3.8, 1.5, 0.3],
        [5.4, 3.4, 1.7, 0.2],
        [5.1, 3.7, 1.5, 0.4],
        [4.6, 3.6, 1. , 0.2],
        [5.1, 3.3, 1.7, 0.5],
        [4.8, 3.4, 1.9, 0.2],
        [5. , 3. , 1.6, 0.2],
        [5. , 3.4, 1.6, 0.4],
        [5.2, 3.5, 1.5, 0.2],
        [5.2, 3.4, 1.4, 0.2],
        [4.7, 3.2, 1.6, 0.2],
        [4.8, 3.1, 1.6, 0.2],
        [5.4, 3.4, 1.5, 0.4],
        [5.2, 4.1, 1.5, 0.1],
        [5

In [4]:
np.unique(y)

array([0, 1, 2])

In [5]:
X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.33, random_state=42)

In [6]:
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')

In [7]:
X.shape

(150, 4)

In [8]:
class IrisData(Dataset):
    def __init__(self, X_train, y_train) -> None:
        super().__init__()
        self.X = torch.from_numpy(X_train)
        self.y = torch.from_numpy(y_train)
        self.y = self.y.type(torch.LongTensor)
        self.len = self.X.shape[0]

    def __getitem__(self, index):
        return self.X[index], self.y[index]
    
    def __len__(self):
        return self.len
        

In [9]:
iris_data = IrisData(X_train=X_train, y_train=y_train)
type(iris_data)

__main__.IrisData

In [10]:
from torch.utils.data import DataLoader
train_loader = DataLoader(dataset=iris_data, batch_size=32)
print(f"X Shape: {iris_data.X.shape}, y shape: {iris_data.y.shape}")

X Shape: torch.Size([100, 4]), y shape: torch.Size([100])


In [11]:
import torch.nn.functional as F
class MulticlassModel(nn.Module):
    def __init__(self, num_features, num_classes, hidden_features) -> None:
        super().__init__()
        self.lin1 = nn.Linear(num_features, hidden_features)
        self.lin2 = nn.Linear(hidden_features, num_classes)
        self.log_softmax = nn.LogSoftmax(dim=1)

    def forward(self, x):
        x = self.lin1(x)
        x = torch.sigmoid(x)
        x = self.lin2(x)
        x = self.log_softmax(x)
        return x

In [12]:
num_features = iris_data.X.shape[1]
hidden = 6
num_classes = len(iris_data.y.unique())

num_features, num_classes



(4, 3)

In [13]:
model = MulticlassModel(num_features, num_classes, hidden)

In [14]:
import torch.optim as optim
criterion = nn.CrossEntropyLoss()
lr = 0.1
optimizer = optim.SGD(model.parameters(), lr=lr)

In [15]:
type(train_loader)

torch.utils.data.dataloader.DataLoader

In [16]:
epocs = 100

losses = []

for epoc in range(epocs):
    for X, y in train_loader:
       

        y_preds = model.forward(X)

        loss = criterion(y_preds, y)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    losses.append(float(loss.data.detach().numpy()))

In [17]:
X_test_torch = torch.from_numpy(X_test)
with torch.no_grad():
    y_test_hat_softmax = model(X_test_torch)
    y_test_hat = torch.max(y_test_hat_softmax.data, 1)

In [18]:
accuracy_score(y_test, y_test_hat.indices)

0.96

In [19]:
from collections import Counter
most_common_cnt = Counter(y_test).most_common()[0][1]
print(f"Naive Classifier: {most_common_cnt / len(y_test) * 100} %")

Naive Classifier: 38.0 %
